In [1]:
import os
import torch
import argparse
import numpy as np
import pandas as pd
from util import * 
from model import *
from pandas import DataFrame
from sklearn.model_selection import KFold
from sklearn.model_selection import  StratifiedKFold

from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader, TensorDataset
from model import NewsDataset, NewDataset_finetuning

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--epoches',              type=int,  default=30,  help='')
parser.add_argument('--batch_size',           type=int,  default=16,  help='')
parser.add_argument('--max_length',           type=int,  default=200, help='')
parser.add_argument('--learning_rate',        type=float, default=1e-4, help="")
parser.add_argument('--model_path',           type=str,  default="../DNA-BERT/3-new-12w-0", help='')
parser.add_argument('--ind_filename',  type=str,  default="../dataset/enhancer_3-mer_DNABERT_ind.txt", help='')
parser.add_argument('--tra_filename',  type=str,  default="../dataset/enhancer_3-mer_DNABERT_tra.txt", help='')

args = parser.parse_args(args=[]) 

# tra_dataloader, val_dataloader = getData(args, split=False, validation=True)
# tra_dataloader = getData(args, split=False, validation=False)
# ind_dataloader = getData(args, split=False, validation=True)
# tra_dataloader, ind_dataloader = getDataLoader_finetuning(args)

In [3]:
def GetDataset(tra_dataset_filename):
    df_raw = pd.read_csv(tra_dataset_filename, sep="\t",header=None,names=["text","label"])
    dataset_text, dataset_label = list(df_raw["text"]), list(df_raw["label"])
    return dataset_text, dataset_label

def getDataloader(dataset_text, dataset_label, args):
    model_path = args.model_path
    max_length = args.max_length
    batch_size = args.batch_size
    
    tokenizer = BertTokenizer.from_pretrained(model_path) 
    dataset_encoding = tokenizer(dataset_text, truncation = True, padding = True, max_length = max_length)

    train_dataset = NewsDataset(dataset_encoding, dataset_label)
    dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
    
    return dataloader

In [4]:
# average [3,4,5,6]
# add L2-nnormalization and average the output of DNABERT in k-mer tokens
# mers = [3, 4, 5, 6]
mers = [3,4,5,6]
lambds = [1e-3, 1e-3, 5e-4, 5e-5]
learning_rates = [5e-5, 2e-5, 5e-5, 1e-5]

val_acc_list, val_mcc_list, val_sn_list, val_sp_list = [], [], [], []

for i in range(len(mers)):
    mer = mers[i]
    lambd = lambds[i]
    learning_rate = learning_rates[i]
    
    val_acc_fold, val_mcc_fold, val_sn_fold, val_sp_fold = [], [], [], []
    
    args.model_path = "../../DNA-BERT/{}-new-12w-0".format(mer)
    args.ind_filename = "../dataset/enhancer_{}-mer_DNABERT_ind.txt".format(mer)
    args.tra_filename = "../dataset/enhancer_{}-mer_DNABERT_tra.txt".format(mer)
    
    dataset_text, dataset_label = GetDataset(args.tra_filename)
    dataset_text = np.array(dataset_text)
    dataset_label = np.array(dataset_label)
    
    folds = 5
    seed = 2022
    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)
    
    for i, (train_index, valid_index) in enumerate(kf.split(dataset_text, dataset_label)):
        
        x_train_flods, y_train_folds = dataset_text[train_index], dataset_label[train_index]
        x_valid_fold, y_valid_fold = dataset_text[valid_index], dataset_label[valid_index]
        
        tra_dataloader = getDataloader(list(x_train_flods), list(y_train_folds), args)
        val_dataloader = getDataloader(list(x_valid_fold), list(y_valid_fold), args)
        
        model = C_Bert_2FC_average.from_pretrained(args.model_path, num_labels=1).to(device)
        
        epoches = args.epoches
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08,)
        scheduler = optim.lr_scheduler.ExponentialLR(optimizer, 0.98)    # 指数衰减损失函数
        
        for epoch in range(epoches):
            tra_acc, tra_loss = train_finetuning_Norm(model, tra_dataloader, optimizer, args, lambd, 2)
            scheduler.step()
            
            val_acc, val_mcc, val_sn, val_sp, val_loss = validation_finetuning(model, val_dataloader)
            
        val_acc_fold.append(val_acc)
        val_mcc_fold.append(val_mcc)
        val_sn_fold.append(val_sn)
        val_sp_fold.append(val_sp)
        
        print("{}-mer, {}-flod, val_acc:{}, val_sn:{}, val_sp:{}, val_mcc:{}".format(mer, i, val_acc, val_sn, val_sp, val_mcc))
        
    val_acc_list.append(val_acc_fold)
    val_mcc_list.append(val_mcc_fold)
    val_sn_list.append(val_sn_fold)
    val_sp_list.append(val_sp_fold)

Some weights of the model checkpoint at ../3-new-12w-0 were not used when initializing C_Bert_2FC_average: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing C_Bert_2FC_average from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing C_Bert_2FC_average from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of C_Bert_2FC_average were not initialized from the model checkpoint at ../3-new-12w-0 and are newly initialized: ['classifier2.weight', 'classifi

3-mer, 0-flod, val_acc:0.7710437710437711, val_sn:0.6805555555555556, val_sp:0.8562091503267973, val_mcc:0.5466805932050623


Some weights of the model checkpoint at ../3-new-12w-0 were not used when initializing C_Bert_2FC_average: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing C_Bert_2FC_average from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing C_Bert_2FC_average from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of C_Bert_2FC_average were not initialized from the model checkpoint at ../3-new-12w-0 and are newly initialized: ['classifier2.weight', 'classifi

3-mer, 1-flod, val_acc:0.797979797979798, val_sn:0.7482993197278912, val_sp:0.8466666666666667, val_mcc:0.598203143194434


Some weights of the model checkpoint at ../3-new-12w-0 were not used when initializing C_Bert_2FC_average: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing C_Bert_2FC_average from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing C_Bert_2FC_average from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of C_Bert_2FC_average were not initialized from the model checkpoint at ../3-new-12w-0 and are newly initialized: ['classifier2.weight', 'classifi

...

6-mer, 2-flod, val_acc:0.7760942760942761, val_sn:0.7612903225806451, val_sp:0.7922535211267606, val_mcc:0.5530258601018122


Some weights of the model checkpoint at ../6-new-12w-0 were not used when initializing C_Bert_2FC_average: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing C_Bert_2FC_average from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing C_Bert_2FC_average from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of C_Bert_2FC_average were not initialized from the model checkpoint at ../6-new-12w-0 and are newly initialized: ['classifier2.weight', 'classifi

6-mer, 3-flod, val_acc:0.7487352445193929, val_sn:0.6816720257234726, val_sp:0.8226950354609929, val_mcc:0.5072088496318126


Some weights of the model checkpoint at ../6-new-12w-0 were not used when initializing C_Bert_2FC_average: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing C_Bert_2FC_average from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing C_Bert_2FC_average from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of C_Bert_2FC_average were not initialized from the model checkpoint at ../6-new-12w-0 and are newly initialized: ['classifier2.weight', 'classifi

6-mer, 4-flod, val_acc:0.7723440134907251, val_sn:0.7437722419928826, val_sp:0.7980769230769231, val_mcc:0.5429650772116754
